In [1]:
import findspark
findspark.init('/Users/admin/spark-3.2.0')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark import  SparkContext,SparkConf
conf = SparkConf().setAll([('spark.driver.host','localhost'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/08 00:44:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/08 00:44:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/08 00:44:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/08 00:44:38 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/12/08 00:44:38 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
import pandas as pd
import numpy as np
import math as m

In [5]:
data=pd.read_csv('data.csv')
data_T=pd.DataFrame(data, columns= ['Object','Property','Value','Source'])
data_T

Object   Property   Value   Source
0     Object1  Property1  179475  Source1
1     Object1  Property3  371893  Source1
2     Object2  Property1  422854  Source1
3     Object2  Property3  324512  Source1
4     Object3  Property1  489481  Source1
..        ...        ...     ...      ...
145   Object6  Property2  319997  Source4
146   Object7  Property2  269456  Source4
147   Object8  Property2  323152  Source4
148   Object9  Property2  286530  Source4
149  Object10  Property2  856218  Source4

[150 rows x 4 columns]

In [6]:
# constitution de dataframe pour le travail
data_T['TSInitial']=0.8
data_T['C_v']=0.0
data_T.head(5)

Object   Property   Value   Source  TSInitial  C_v
0  Object1  Property1  179475  Source1        0.8  0.0
1  Object1  Property3  371893  Source1        0.8  0.0
2  Object2  Property1  422854  Source1        0.8  0.0
3  Object2  Property3  324512  Source1        0.8  0.0
4  Object3  Property1  489481  Source1        0.8  0.0

In [7]:
# Convertissons notre dataFrame en RDD
data_T.to_csv('data_T.csv',index=False)
read_ = sc.textFile("data_T.csv")
# Spliter selon les lignes de notre dataFrame
read_rdd=read_.map(lambda line: line.split(",")).map(lambda line:(line[0],line[1],line[2],line[3],line[4],line[5])).filter(lambda x: x[0]!='Object')
# Lecture de notre RDD
read_rdd.take(5)

[('Object1', 'Property1', '179475', 'Source1', '0.8', '0.0'),
 ('Object1', 'Property3', '371893', 'Source1', '0.8', '0.0'),
 ('Object2', 'Property1', '422854', 'Source1', '0.8', '0.0'),
 ('Object2', 'Property3', '324512', 'Source1', '0.8', '0.0'),
 ('Object3', 'Property1', '489481', 'Source1', '0.8', '0.0')]

In [8]:
# Definition de la matice de similarite
def similarity(dataframe):
    dict_sim = {}
    for key, df in dataframe.groupby(by=['Object','Property']):
        Values = df['Value'].unique()
        row = key[0]+key[1]
        for  i in range(len(Values)):
            w1 = Values[i]
            for  u in range(len(Values)):
                w2 = Values[u]
                sim = 1
                if w1!=w2:
                    t = abs(w1-w2)
                    sim = 1/t
                dict_sim[row+str(w1)+str(w2)] = sim
                dict_sim[row+str(w2)+str(w1)] = sim
    return dict_sim

In [9]:
# Declaration des variables globales
global data_T
data_T=pd.read_csv('data_T.csv')
global rho
rho=0.7
global lam
lam=0.5
global dict_sim
global read_rdd



In [10]:
dict_sim=similarity(data)


# la fonction Map
# (input x est un quatriplet constitutue de (object,property,valeur, TSinitial) et le renvoie 
# la cle(object,property,valeur) et la valeur sigma_v)
def MapFunction(x):
    return ((x[0],x[1],x[2]), (-m.log(1-float(x[4])), [(x[3],x[4])] ) )
def ReduceFunction(x,y):
    return x[0]+y[0],x[1]+y[1]
def Mapelement(x):
    return (x[0][0],x[0][1]),[(x[0][2],x[1][0],x[1][1])]
def ReduceShuffling(x,y):
    return x+y

def reduceFonction(x):

    out=(x[0],[])
    for v in x[1]:
        c_v=v[1]
        l=0
        for v_ in x[1]:
            if v[0]!=v_[0]:
                Key_sim=x[0][0]+x[0][1]+str(v[0])+str(v_[0])
                c_v+=rho*(v_[1]*dict_sim[Key_sim])

        CV=1/(1+np.exp(-lam*c_v))

        out[1].append((v[0],CV,v[2]))

    return out
# rdd_cv = sc.parallelize(transf.map(reduceFonction).map(mapT).reduceByKey(lambda x,y: x+y ).collect()[0][1])
def mapT(x):
    out=[]
    for v in x[1]:
        for s in v[2]:
            out.append((x[0][0],x[0][1],v[0],s[0],s[1],v[1]))
    return (1,out)
def mapTS(x):
    return x[3],x[5]
def mapReconstruit(x):
    ts = 0
    for s in source_ts:
        if x[3]==s[0]:
            ts = s[1]
            break
    return x[0],x[1],x[2],x[3],ts,x[5]


def mapTS_(x):
    return x[3],x[4]


def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [11]:
import time as t
def truthF(data_T):
    read_ = sc.textFile("data_T.csv")
    # Spliter selon les lignes de notre dataFrame
    read_rdd=read_.map(lambda line: line.split(",")).map(lambda line:(line[0],line[1],line[2],line[3],line[4],line[5])).filter(lambda x: x[0]!='Object')
    ni=5
    trustold = 0.00001
    global source_ts 
    source_ts = []
    start = t.time()
    nb_iter_do = 0
    for i in range(ni):
        
        ts1 = list(read_rdd.map(mapTS_).reduceByKey(lambda x,y: x).sortBy(lambda x: x[0]).map(lambda x: float(x[1])).collect())
        
        fonc=read_rdd.map(MapFunction).reduceByKey(ReduceFunction)
        foncajust=fonc.map(Mapelement).reduceByKey(ReduceShuffling).map(reduceFonction)
        
        transf=sc.parallelize(foncajust.map(mapT).reduceByKey(lambda x,y: x+y ).collect()[0][1])
        
        source_ts = list(transf.map(mapTS).groupByKey().mapValues(lambda x: sum(x) / len(x)).collect())
        
        read_rdd = transf.map(mapReconstruit)
        
        ts2 = list(read_rdd.map(mapTS_).reduceByKey(lambda x,y: x).sortBy(lambda x: x[0]).map(lambda x: float(x[1])).collect())
        
        print('Itération : ',i+1)
        print(1-abs(cos_sim(ts1,ts2)))
        nb_iter_do = i+1
        if 1-abs(cos_sim(ts1,ts2)) < trustold:
            break
    end = t.time()
    
    
    return read_rdd,end-start,nb_iter_do
    
    
    

In [14]:
out = truthF(data_T)
out

Itération :  1
0.001083795431621959
Itération :  2
0.0003308620801112383
Itération :  3
7.63023128548257e-05
Itération :  4
1.5614884195214174e-05
Itération :  5
3.0793252884242506e-06


(PythonRDD[626] at RDD at PythonRDD.scala:53, 3.794365167617798, 5)

In [15]:
out[0].take(2)

[('Object1',
  'Property3',
  '745886',
  'Source2',
  0.6724265387159437,
  0.6370540400061349),
 ('Object1',
  'Property3',
  '953423',
  'Source4',
  0.6724225416310106,
  0.6370522329664958)]

In [16]:
def mapGetResult(x):
    return ((x[0],x[1]),(x[2],x[5]))
def reduceGetResult(x,y):
    return y
def reduceVote(x,y):
    if x[1] > y[1]:
        return x
    return y

In [17]:
out[0].map(mapGetResult)

PythonRDD[628] at RDD at PythonRDD.scala:53

In [18]:
list_A =out[0].map(mapGetResult).reduceByKey(reduceVote).map(lambda x: (x[0][0],x[0][1],x[1][0])).collect()

/Users/admin/spark-3.2.0/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [19]:
list_A

[('Object1', 'Property3', '371893'),
 ('Object4', 'Property3', '292202'),
 ('Object7', 'Property2', '269456'),
 ('Object9', 'Property2', '286530'),
 ('Object10', 'Property2', '631083'),
 ('Object10', 'Property1', '786290'),
 ('Object7', 'Property1', '337793'),
 ('Object9', 'Property1', '663325'),
 ('Object2', 'Property3', '324512'),
 ('Object5', 'Property3', '205622'),
 ('Object4', 'Property1', '433866'),
 ('Object5', 'Property1', '230773'),
 ('Object7', 'Property3', '361249'),
 ('Object1', 'Property1', '179475'),
 ('Object9', 'Property3', '895064'),
 ('Object1', 'Property2', '168208'),
 ('Object5', 'Property2', '388816'),
 ('Object10', 'Property3', '724616'),
 ('Object2', 'Property2', '432090'),
 ('Object4', 'Property2', '475263'),
 ('Object2', 'Property1', '422854'),
 ('Object3', 'Property3', '122120'),
 ('Object6', 'Property3', '327221'),
 ('Object8', 'Property1', '433516'),
 ('Object8', 'Property2', '323152'),
 ('Object8', 'Property3', '977818'),
 ('Object3', 'Property1', '489481')

In [20]:
list_B = read_rdd.collect()

In [21]:
data_truth=pd.read_csv('data_truth.csv')
data_truth_=pd.DataFrame(data_truth, columns= ['Object','Property','Value'])
data_truth_.to_csv('data_truth_.csv',index=False)
read_truth = sc.textFile("data_truth_.csv")
truth_rdd=read_truth.map(lambda line: line.split(",")).map(lambda line:(line[0],line[1],line[2]))

In [22]:
list_truth=truth_rdd.collect()

In [23]:
def function(list_A,list_B):
    binary=[]
    search_list = [tuple(li[:-3]) for li in list_B]
    for ser_item in search_list:
        if ser_item in list_A:
            binary.append(1)
        else:
            binary.append(0)
    return binary
        
    

In [24]:
res = function(list_A, list_B)
res_truth=function(list_truth, list_B)
res
res_truth

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0]

In [25]:
!pip install scikit-learn
import sklearn

In [26]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [31]:
precision_score(res_truth,res)

0.9

In [28]:
accuracy_score(res_truth,res)

0.9133333333333333

In [29]:
recall_score(res_truth,res)

0.9130434782608695

In [30]:
f1_score(res_truth,res)

0.9064748201438848